In [ ]:
import cv2
import time
import csv
from datetime import datetime

# Load Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# OpenZone thresholds
EYE_CONTACT_ZONE = (200, 150, 250, 200)  # Ideal zone
POSTURE_THRESHOLD_NEAR = 250  # Face too close to camera

# Logging vars
eye_contact_start = None
last_break_reminder = None
log_file = "eyezone_log.csv"

# Create/open log file
with open(log_file, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Timestamp", "Event"])

cap = cv2.VideoCapture(0)
cv2.namedWindow("EyeZone")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.2, 6)

    posture_warning = False
    eye_contact = False

    for (x, y, w, h) in faces:
        # Draw face box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Check posture
        if w > POSTURE_THRESHOLD_NEAR:
            cv2.putText(frame, "Move Back! Posture Bad!", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            posture_warning = True
            with open(log_file, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Posture Too Close"])

        # Check if eyes are in EyeZone
        cx, cy = x + w // 2, y + h // 2
        (zx, zy, zw, zh) = EYE_CONTACT_ZONE
        if zx < cx < zx + zw and zy < cy < zy + zh:
            eye_contact = True

        break  # Use only first face

    # Eye contact timer logic
    now = time.time()
    if eye_contact:
        if eye_contact_start is None:
            eye_contact_start = now
        elif now - eye_contact_start > 10 * 60:  # 10 minutes
            if last_break_reminder is None or now - last_break_reminder > 60:
                cv2.putText(frame, "Take a Break! Eye strain warning.", (30, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                            (0, 165, 255), 2)
                with open(log_file, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Break Reminder"])
                last_break_reminder = now
    else:
        eye_contact_start = None

    # Draw EyeZone box
    (zx, zy, zw, zh) = EYE_CONTACT_ZONE
    cv2.rectangle(frame, (zx, zy), (zx + zw, zy + zh), (0, 255, 0), 2)
    cv2.putText(frame, "Eye Contact Zone", (zx, zy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    cv2.imshow("EyeZone", frame)

    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import time
import csv
from datetime import datetime
from playsound import playsound
import threading

# Constants
REAL_FACE_WIDTH_CM = 16.0
FOCAL_LENGTH = 500
MIN_DISTANCE_CM = 25
BREAK_TIME_SEC = 10 * 60  # 10 minutes

# Eye contact zone
EYE_CONTACT_ZONE = (200, 150, 250, 200)

# Load Haar Cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Logging
eye_contact_start = None
last_break_reminder = None
log_file = "eyezone_log.csv"

# Create/open log
with open(log_file, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Timestamp", "Event"])

def play_beep():
    threading.Thread(target=playsound, args=("C:\Users\user\Downloadsbraam-sound-effect-339460.mp3",), daemon=True).start()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 6)

    eye_contact = False

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Distance
        distance_cm = (REAL_FACE_WIDTH_CM * FOCAL_LENGTH) / w
        cv2.putText(frame, f"Distance: {int(distance_cm)} cm", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        if distance_cm < MIN_DISTANCE_CM:
            cv2.putText(frame, "Too Close to Screen!", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            play_beep()
            with open(log_file, "a", newline="") as f:
                csv.writer(f).writerow([datetime.now(), "Too Close (Distance Alert)"])

        # Eye contact
        cx, cy = x + w // 2, y + h // 2
        (zx, zy, zw, zh) = EYE_CONTACT_ZONE
        if zx < cx < zx + zw and zy < cy < zy + zh:
            eye_contact = True

        break

    now = time.time()
    if eye_contact:
        if eye_contact_start is None:
            eye_contact_start = now
        elif now - eye_contact_start > BREAK_TIME_SEC:
            if last_break_reminder is None or now - last_break_reminder > 60:
                cv2.putText(frame, "Take a Break!", (30, 450), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 165, 255), 2)
                play_beep()
                with open(log_file, "a", newline="") as f:
                    csv.writer(f).writerow([datetime.now(), "Break Reminder"])
                last_break_reminder = now
    else:
        eye_contact_start = None

    # Draw Eye Contact Zone
    (zx, zy, zw, zh) = EYE_CONTACT_ZONE
    cv2.rectangle(frame, (zx, zy), (zx + zw, zy + zh), (0, 255, 0), 2)
    cv2.putText(frame, "Eye Contact Zone", (zx, zy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    cv2.imshow("EyeZone", frame)

    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import time
import csv
import os
from datetime import datetime
from playsound import playsound
import threading

# Constants
REAL_FACE_WIDTH_CM = 16.0
FOCAL_LENGTH = 500
MIN_DISTANCE_CM = 25
BREAK_TIME_SEC = 10 * 60  # 10 minutes
POSTURE_THRESHOLD_NEAR = 150  # face width in pixels

# Eye contact zone (center box area)
EYE_CONTACT_ZONE = (200, 150, 250, 200)

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Logging setup
log_file = "eyezone_log.csv"
if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as f:
        csv.writer(f).writerow(["Timestamp", "Event"])

# ✅ Use your sound file (fixed path)
sound_path = r"C:\Users\user\Downloads\braam-sound-effect-339460.mp3"

# Sound alert function
def play_beep():
    try:
        threading.Thread(target=playsound, args=(sound_path,), daemon=True).start()
    except Exception as e:
        print("Sound Error:", e)

# Timers
eye_contact_start = None
last_break_reminder = None

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 6)

    eye_contact = False

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Distance calculation
        distance_cm = (REAL_FACE_WIDTH_CM * FOCAL_LENGTH) / w
        cv2.putText(frame, f"Distance: {int(distance_cm)} cm", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        if distance_cm < MIN_DISTANCE_CM:
            cv2.putText(frame, "Too Close to Screen!", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            play_beep()
            with open(log_file, "a", newline="") as f:
                csv.writer(f).writerow([datetime.now(), "Too Close (Distance Alert)"])

        # Posture Check via Face Width
        if w > POSTURE_THRESHOLD_NEAR:
            cv2.putText(frame, "Bad Posture Detected!", (30, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 100, 255), 2)
            play_beep()
            with open(log_file, "a", newline="") as f:
                csv.writer(f).writerow([datetime.now(), "Bad Posture (Too Close Face Width)"])

        # Eye contact zone
        cx, cy = x + w // 2, y + h // 2
        (zx, zy, zw, zh) = EYE_CONTACT_ZONE
        if zx < cx < zx + zw and zy < cy < zy + zh:
            eye_contact = True

        break

    now = time.time()
    if eye_contact:
        if eye_contact_start is None:
            eye_contact_start = now
        elif now - eye_contact_start > BREAK_TIME_SEC:
            if last_break_reminder is None or now - last_break_reminder > 60:
                cv2.putText(frame, "Take a Break!", (30, 450), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 165, 255), 2)
                play_beep()
                with open(log_file, "a", newline="") as f:
                    csv.writer(f).writerow([datetime.now(), "Break Reminder"])
                last_break_reminder = now
    else:
        eye_contact_start = None

    # Eye zone overlay
    (zx, zy, zw, zh) = EYE_CONTACT_ZONE
    cv2.rectangle(frame, (zx, zy), (zx + zw, zy + zh), (0, 255, 0), 2)
    cv2.putText(frame, "Eye Contact Zone", (zx, zy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    cv2.imshow("EyeZone – Posture + Eye Distance Monitor", frame)

    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
